In this notebook, we want to predict gene regulatory factors relative to the healthy single-cell reference, that improve deconvolution performance. Therefore we apply module 3 with the hyperparameters we found in `05b_Hyperparameter_Search.ipynb`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import deconomix as dcx

In [ ]:
PREFIX = '05c_'
l2_LumA  = 1e-8
l2_LumB  = 1e-10
l2_Her2  = 1e-07
l2_Basal = 1e-07